In [2]:
import requests as rq
from bs4 import BeautifulSoup

url = 'https://finance.naver.com/sise/sise_deposit.nhn'
data = rq.get(url)
data_html = BeautifulSoup(data.content)
parse_day = data_html.select_one(
    'div.subtop_sise_graph2 > ul.subtop_chart_note > li > span.tah').text

print(parse_day)


  |  2023.06.27


In [4]:
import re

biz_day = re.findall('[0-9]+', parse_day)
biz_day = ''.join(biz_day)

print(biz_day)

20230627


In [5]:
from io import BytesIO
import pandas as pd

gen_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
gen_otp_stk = {
    'mktId': 'STK',
    'trdDd': biz_day,
    'money': '1',
    'csvxls_isNo': 'false',
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/standard/MDCSTAT03901'
}
headers = {'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader'}
otp_stk = rq.post(gen_otp_url, gen_otp_stk, headers=headers).text

print(otp_stk)

PxldVNq82TWyLSb3hI2XSQ0b4byJWSqWVxgrlBKCrRcRtSksuLS7Bnxpl86F7dAOkunw9BBwugQaSjGAcH15eWG2Kx8fxmE/1AYcSZEH3estBgM+EFJCxYg3zco1gIgRZqIo4cIzoURnTI8+MmkJ4m8vFLhSKmM794gFu+ThsO31lY4woqehX8j6OlXFDcfHdV4NbYo4+D2Rwcfj24VnU3Zpq3ik/Dyw3FdyOXhJkBI=


In [6]:
down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
down_sector_stk = rq.post(down_url, {'code': otp_stk}, headers=headers)
sector_stk = pd.read_csv(BytesIO(down_sector_stk.content), encoding='EUC-KR')

sector_stk.head()

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액
0,095570,AJ네트웍스,KOSPI,서비스업,4525,65,1.46,211870884875
1,006840,AK홀딩스,KOSPI,기타금융,19400,-330,-1.67,257002683400
2,027410,BGF,KOSPI,기타금융,3875,15,0.39,370902565125
3,282330,BGF리테일,KOSPI,유통업,182300,-1700,-0.92,3150856063800
4,138930,BNK금융지주,KOSPI,기타금융,6910,100,1.47,2252212549860


In [7]:
gen_otp_ksq = {
    'mktId': 'KSQ',  # 코스닥 입력
    'trdDd': biz_day,
    'money': '1',
    'csvxls_isNo': 'false',
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/standard/MDCSTAT03901'
}
otp_ksq = rq.post(gen_otp_url, gen_otp_ksq, headers=headers).text

down_sector_ksq = rq.post(down_url, {'code': otp_ksq}, headers=headers)
sector_ksq = pd.read_csv(BytesIO(down_sector_ksq.content), encoding='EUC-KR')

sector_ksq.head()

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액
0,060310,3S,KOSDAQ,기계·장비,2820,-75,-2.59,136873330440
1,054620,APS,KOSDAQ,금융,9840,-120,-1.20,200679134640
2,265520,AP시스템,KOSDAQ,반도체,21200,50,0.24,323966125200
3,211270,AP위성,KOSDAQ,통신장비,13290,-160,-1.19,200443820160
4,126600,BGF에코머티리얼즈,KOSDAQ,화학,6110,30,0.49,249709675540


In [8]:
krx_sector = pd.concat([sector_stk, sector_ksq]).reset_index(drop=True)
krx_sector['종목명'] = krx_sector['종목명'].str.strip()
krx_sector['기준일'] = biz_day

krx_sector.head()

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액,기준일
0,095570,AJ네트웍스,KOSPI,서비스업,4525,65,1.46,211870884875,20230627
1,006840,AK홀딩스,KOSPI,기타금융,19400,-330,-1.67,257002683400,20230627
2,027410,BGF,KOSPI,기타금융,3875,15,0.39,370902565125,20230627
3,282330,BGF리테일,KOSPI,유통업,182300,-1700,-0.92,3150856063800,20230627
4,138930,BNK금융지주,KOSPI,기타금융,6910,100,1.47,2252212549860,20230627


In [54]:
import requests as rq
from io import BytesIO
import pandas as pd

gen_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
gen_otp_data = {
    'searchType': '1',
    'mktId': 'ALL',
    'trdDd': biz_day,
    'csvxls_isNo': 'false',
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/standard/MDCSTAT03501'
}
headers = {'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader'}
otp = rq.post(gen_otp_url, gen_otp_data, headers=headers).text

down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
krx_ind = rq.post(down_url, {'code': otp}, headers=headers)

krx_ind = pd.read_csv(BytesIO(krx_ind.content), encoding='EUC-KR')
krx_ind['종목명'] = krx_ind['종목명'].str.strip()
krx_ind['기준일'] = biz_day

krx_ind.head()

,종목코드,종목명,종가,대비,등락률,EPS,PER,선행 EPS,선행 PER,BPS,PBR,주당배당금,배당수익률,기준일
0,060310,3S,2820,-75,-2.59,33.0,85.45,NaN,NaN,829.0,3.40,0,0.00,20230627
1,095570,AJ네트웍스,4525,65,1.46,201.0,22.51,630.0,7.18,8076.0,0.56,270,5.97,20230627
2,006840,AK홀딩스,19400,-330,-1.67,NaN,NaN,NaN,NaN,41948.0,0.46,200,1.03,20230627
3,054620,APS,9840,-120,-1.20,505.0,19.49,NaN,NaN,10864.0,0.91,0,0.00,20230627
4,265520,AP시스템,21200,50,0.24,5463.0,3.88,3830.0,5.54,17980.0,1.18,270,1.27,20230627


In [44]:
len(krx_ind)

2550

In [15]:
##일단 안씀
top_5_high_pbr = krx_ind.sort_values(by='PBR', ascending=False).head(5)
top_5_high_pbr = top_5_high_pbr[['종목코드', '종목명', 'PBR', '기준일']].reset_index(drop=True)
print(top_5_high_pbr)

     종목코드      종목명      PBR       기준일
0  417860      오브젠  1191.18  20230627
1  015540  쎌마테라퓨틱스    92.90  20230627
2  411080      샌즈랩    58.13  20230627
3  217330     싸이토젠    51.38  20230627
4  178780    일월지엠엘    47.79  20230627


In [55]:
top_5_high_BPS = krx_ind.sort_values(by='BPS', ascending=False).head(5)
top_5_high_BPS = top_5_high_BPS[['종목코드', '종목명', 'BPS', '기준일']].reset_index(drop=True)
print(top_5_high_BPS)

     종목코드   종목명        BPS       기준일
0  003240  태광산업  4906107.0  20230627
1  000670    영풍  2272640.0  20230627
2  003920  남양유업  1079345.0  20230627
3  001460   BYC   825735.0  20230627
4  002380   KCC   695902.0  20230627


In [17]:
##가치투자형을 위한 PER 상위 5개 종목
top_5_low_per = krx_ind.dropna(subset=['PER']).sort_values(by='PER').head(5)
top_5_low_per = top_5_low_per[['종목코드', '종목명','PER', '기준일']].reset_index(drop=True)
print(top_5_low_per)

     종목코드     종목명   PER       기준일
0  106240  파인테크닉스  0.31  20230627
1  000950      전방  0.48  20230627
2  104480  티케이케미칼  0.77  20230627
3  035080   그래디언트  0.86  20230627
4  004870  티웨이홀딩스  0.91  20230627


In [20]:
##안정형을 위한 배당수익률 상위 5개 종목
top_5_high_allocation = krx_ind.sort_values(by='배당수익률', ascending=False).head(5)
top_5_high_allocation = top_5_high_allocation[['종목코드', '종목명', '배당수익률', '기준일']].reset_index(drop=True)
print(top_5_high_allocation)

     종목코드     종목명  배당수익률       기준일
0  115390     락앤락  32.88  20230627
1  003120    일성신약  14.75  20230627
2  012700    리드코프  11.73  20230627
3  010955  S-Oil우  10.60  20230627
4  377740   바이오노트  10.44  20230627


In [27]:
from datetime import datetime, timedelta

current_date = datetime.now().date()
year_ago_date = current_date - timedelta(days=730)
year_ago_date_str = year_ago_date.strftime('%Y%m%d')

print(year_ago_date_str)


20210629


In [41]:
import requests as rq
from io import BytesIO
import pandas as pd
import numpy as np

# 1년 전 데이터 가져오기
gen_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
gen_otp_data = {
    'searchType': '1',
    'mktId': 'ALL',
    'trdDd': year_ago_date_str,
    'csvxls_isNo': 'false',
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/standard/MDCSTAT03501'
}
headers = {'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader'}
otp = rq.post(gen_otp_url, gen_otp_data, headers=headers).text

down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
krx_ind_year_ago = rq.post(down_url, {'code': otp}, headers=headers)

krx_ind_year_ago = pd.read_csv(BytesIO(krx_ind_year_ago.content), encoding='EUC-KR')
krx_ind_year_ago['종목명'] = krx_ind_year_ago['종목명'].str.strip()
krx_ind_year_ago['기준일'] = year_ago_date_str

# 현재 데이터 가져오기
gen_otp_data['trdDd'] = biz_day
otp = rq.post(gen_otp_url, gen_otp_data, headers=headers).text

krx_ind = rq.post(down_url, {'code': otp}, headers=headers)
krx_ind = pd.read_csv(BytesIO(krx_ind.content), encoding='EUC-KR')
krx_ind['종목명'] = krx_ind['종목명'].str.strip()
krx_ind['기준일'] = biz_day

# EPS 성장률 계산
eps_growth_rate = ((krx_ind['EPS'] - krx_ind_year_ago['EPS']) / krx_ind_year_ago['EPS']) * 100
krx_ind['EPS 성장률'] = eps_growth_rate.where(eps_growth_rate.notnull(), np.nan)

peg = krx_ind['PER'] / krx_ind['EPS 성장률']
krx_ind['PEG'] = peg.where(peg.notnull(), np.nan)

krx_ind.head()

,종목코드,종목명,종가,대비,등락률,EPS,PER,선행 EPS,선행 PER,BPS,PBR,주당배당금,배당수익률,기준일,EPS 성장률,PEG
0,060310,3S,2820,-75,-2.59,33.0,85.45,NaN,NaN,829.0,3.40,0,0.00,20230627,NaN,NaN
1,095570,AJ네트웍스,4525,65,1.46,201.0,22.51,630.0,7.18,8076.0,0.56,270,5.97,20230627,NaN,NaN
2,006840,AK홀딩스,19400,-330,-1.67,NaN,NaN,NaN,NaN,41948.0,0.46,200,1.03,20230627,NaN,NaN
3,054620,APS,9840,-120,-1.20,505.0,19.49,NaN,NaN,10864.0,0.91,0,0.00,20230627,NaN,NaN
4,265520,AP시스템,21200,50,0.24,5463.0,3.88,3830.0,5.54,17980.0,1.18,270,1.27,20230627,214.68894,0.018073


In [42]:
## 공격형을 위한 PEG 상위종목 5개
top_5_low_peg = krx_ind.dropna(subset=['PEG']).sort_values(by='PEG').head(5)
top_5_low_peg = top_5_low_peg[['종목코드', '종목명','PEG', '기준일']].reset_index(drop=True)
print(top_5_low_peg)

     종목코드    종목명        PEG       기준일
0  217820  원익피앤이 -90.933726  20230627
1  101360  에코앤드림 -62.320295  20230627
2  267320   나인테크 -32.974553  20230627
3  048770    TPC -17.278144  20230627
4  420770   기가비스 -12.143262  20230627


In [50]:
import requests as rq
from io import BytesIO
import pandas as pd

gen_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
gen_otp_data = {
    'searchType': '1',
    'mktId': 'ALL',
    'trdDd': biz_day,
    'csvxls_isNo': 'false',
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/standard/MDCSTAT03501'
}
headers = {'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader'}
otp = rq.post(gen_otp_url, gen_otp_data, headers=headers).text

down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
krx_ind = rq.post(down_url, {'code': otp}, headers=headers)

krx_ind = pd.read_csv(BytesIO(krx_ind.content), encoding='EUC-KR')
krx_ind['종목명'] = krx_ind['종목명'].str.strip()
krx_ind['기준일'] = biz_day

# PER 수치를 기준으로 오름차순 정렬하여 상위 5개 종목 선택
top_5_low_per = krx_ind.dropna(subset=['PER']).sort_values(by='PER').head(5)

# 종목코드와 종목명으로 이루어진 데이터프레임 생성
top_5_low_per = top_5_low_per[['종목코드', '종목명']].reset_index(drop=True)

# 데이터프레임 출력
print(top_5_low_per)


     종목코드     종목명
0  106240  파인테크닉스
1  000950      전방
2  104480  티케이케미칼
3  035080   그래디언트
4  004870  티웨이홀딩스
